In [1]:
######################################## Prediction ###############################################

In [ ]:
#generating patches from predictions image


def prediction_patch(X,patch_size=320):
    
    
    patch_paths=[]
    patch_count=0
    print(X.shape)



    n_vertical_patches=math.ceil(X.shape[0]/patch_size)
    n_horizontal_patches=math.ceil(X.shape[1]/patch_size)
    print("no. of horizontal patches:",n_horizontal_patches,"    no. of vertical patches:",n_vertical_patches)
    print(X.shape)

    for k in range(0,n_vertical_patches):
        for l in range(0,n_horizontal_patches):

          temp_x=X[k*patch_size:(k+1)*patch_size,l*patch_size:(l+1)*patch_size,:]
          
          print("loading entry "+str(patch_count)+"......")
          #print(k*patch_size,(k+1)*patch_size,l*patch_size,(l+1)*patch_size)
          
          np.save("Image_patch_test/img_patch"+str(patch_count)+".npy",temp_x)
          
          patch_paths.append("Image_patch_test/img_patch"+str(patch_count)+".npy")
          patch_count=patch_count+1
              
    return patch_count,patch_paths                                                              #returns the no. of patches generated and the corresponding patch paths

In [ ]:
############################### Prediction ############################################

#returns the prediction mask for a given image (path)

def get_prediction(image_path):
  
  #image_path is the path of the image over which prediction has to be generated
  
  if os.path.exists(image_path):
      
      #loading the image
      img=np.array(imageio.imread(image_path,format="tiff"))
      shape=img.shape
      print(img.shape,end=" ")
      
      #preprocessing the image
      img = normalize(img)
      img=resize(img)
      print("shape of image:",img.shape)
      img=np.array(img)
      
      n_patches,patch_paths=prediction_patch(img)
      
      count=0
      patches=[]
      predict_paths=[]
      
      for path in patch_paths:
        patch=np.load(path)
        patches.append(patch)
        
      patches=np.array(patches)  
      predictions=new_model.predict(patches,batch_size=4)
      
      for prediction in predictions:
        
        prediction=np.argmax(prediction,axis=2)
        image_class=get_image_from_classes(prediction)
        
        np.save("Prediction_patches/patch"+str(count)+".npy",image_class)
        predict_paths.append("Prediction_patches/patch"+str(count)+".npy")
        count=count+1

        
      image=restore_image_from_patches(predict_paths,shape)
      return image
  
  return "error"

In [ ]:
## function to generate rgb image from previosly generated patches

def restore_image_from_patches(patch_paths,image_shape,patch_size=320):
  
  count=0
  img_height,img_width,n_channels=image_shape
  n_channels=3
  
  #calculating the extended height and width based on true shape
  
  n_vertical_patches=math.ceil(img_height/patch_size)
  n_horizontal_patches=math.ceil(img_width/patch_size)
  extended_height = patch_size * n_vertical_patches
  extended_width = patch_size * n_horizontal_patches
  ext_image = np.zeros(shape=(extended_height, extended_width, n_channels), dtype=np.float32)
  
  print(n_vertical_patches,n_horizontal_patches)
  print(extended_height,extended_width)
  
  
  #joining patches to form the actual image
  
  for k in range(0,n_vertical_patches):
      for l in range(0,n_horizontal_patches):
        print(count)
        patch=np.load(patch_paths[count])
        #print(patch.shape)
        print(k*patch_size,(k+1)*patch_size,l*patch_size,(l+1)*patch_size)
        print("loading entry "+str(count)+"......")
        ext_image[k*patch_size:(k+1)*patch_size,l*patch_size:(l+1)*patch_size,:]=patch[:,:,:]
        count=count+1
  
  image=ext_image[:img_height,:img_width,:n_channels]
  
  return image

In [ ]:
!mkdir Image_patch_test
!mkdir Prediction_patches

In [ ]:
path="./sat/4.tif"                                                         #give path of the test image  for which prediction is to be generated

im=get_prediction(path)

print(im.shape)

In [ ]:
################################Calculating metric scores ######################################################3

from mlxtend.evaluate import confusion_matrix 
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics  import cohen_kappa_score


In [ ]:
def slice(img):
    mask=[]
    for i in range(9):
         mask.append(img[:,:,i])
    return mask 

In [ ]:
#function to calculate the kappa coefficient per class on a training  images and their corresponding predictions 


def get_output():
    path='./' 
    i=1
    scores=[]
    while True:
        sat_path=path+'sat/'+str(i)+'.tif'
        if os.path.exists(sat_path):
            K_score=[]
            img=np.array(imageio.imread(sat_path,format="tiff"))
            print(img.shape,end=" ")
           
            print(img.shape)
            gt_path = path+'gt/'+str(i)+'.tif'
            mask = cv2.imread(gt_path)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
           
            mask = get_mask(img,mask)
            mask = slice(mask)
            pred_mask=get_prediction(sat_path)
            pred_mask=get_mask(img,pred_mask)
            pred_mask=slice(pred_mask)
            for j in range(9):
                 K_score.append(cohen_kappa_score(mask[j].argmax(axis=0),pred_mask[j].argmax(axis=0),labels=None))
                 print(K_score[j])
            scores.append(K_score)
        else:
          print("end of loop")
          break
        i=i+1  
    return K_score        

In [ ]:
### kappa coefficient calculated on training images
### the kappa scores generated per class can be found in the output below for each of the 14 training images respectively
### the scores are printed in a series of 9 values, after loading the patches for a particular images 
### the scores are for classes 0,1,2,3,4,5,6,7,8 as printes respectively

K_score=get_output()

In [ ]:
#function to calculate the kappa coefficient per class on a training  images and their corresponding predictions 

def confusion_matrix1():
    path="./"

    i=1

    stack=[]

    while True:
        sat_path=path+'sat/'+str(i)+'.tif'
        if os.path.exists(sat_path):
            img=np.array(imageio.imread(sat_path,format="tiff"))
            print(img.shape)
            gt_path = path+'gt/'+str(i)+'.tif'
            mask = cv2.imread(gt_path)
            #print("mask before getting classes",mask.shape)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            mask=get_classes_from_image(mask)
            #print("mask after getting classes",mask.shape)
            #mask = resize(mask)
            pred_img=get_prediction(sat_path)
            #print("pred before getting classes",pred_img.shape)
            pred_img=get_classes_from_image(pred_img)
            #print("pred after getting classes",pred_img.shape)
            mask=mask.flatten()
            #print("mask after flatten",mask.shape)
            pred_img=pred_img.flatten()
            #print("pred after flatten",pred_img.shape)
            
            
            y=confusion_matrix(mask, pred_img,binary=False)
            print(y,y.shape)
            stack.append(y)
        else: 
          break     
        i=i+1    
    return stack      

In [ ]:
### the confusion matrix for each training image is printed after loading the data for each image
### the horizontal rows denote the actual classes while the columns show the predictions for each class(classes can be inferred from the corresponding indices i.e. index 0 represents class 0)
cm=confusion_matrix1()